<a href="https://colab.research.google.com/github/Samikshya0620/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
# !pip install chromadb transformers langchain sentence_transformers docarray xformers faiss-cpu --quiet

In [73]:
import warnings

# Filter out the LangChainDeprecationWarning
warnings.filterwarnings("ignore")


In [100]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "********"

In [75]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(model_id="google/flan-t5-large", task="text2text-generation", model_kwargs={"temperature": 0, "max_length": 200})

In [83]:
from google.colab import files
uploaded = files.upload()

Saving Image Processing with CNN.pdf to Image Processing with CNN (1).pdf


In [8]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.1 MB/s eta 0:00:00


In [85]:
# !pip install pypdf

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Image Processing with CNN.pdf")
# df = loader.load_and_split()
documents = loader.load()

In [86]:
documents

[Document(page_content='Image Processing with CNN (ResNet):  \nThe process begins with image input, which is fed into a Convolutional Neural Network (CNN)  \nmodel, specifically ResNet (Residual Neural Network), known for its effectiveness in image  \nrecognition tasks.  \nResNet extracts hi gh-level features from the image, capturing its visual content in a structured  \nmanner.  \n \nCaption Generation with Transformer:  \nNext, the extracted image features are passed to a Transformer -based model for natural  \nlanguage processing.  \nThe Transformer model generates descriptive nepali captions based on the features extracted  \nfrom the image. This step involves understanding the context and content of the image to  \nproduce coherent and relevant descriptions.  \n \nText -to-Speech (TTS) Conversion:  \nThe generated capt ion, now represented as a textual string, is then passed to a pre -trained TTS  \nmodel.  \nThis TTS model converts the textual input into synthesized speech, effe

In [93]:
from langchain.chains.question_answering import load_qa_chain

# we are specifying that OpenAI is the LLM that we want to use in our chain
chain = load_qa_chain(llm=llm)
query = 'what does the transformer do?'
response = chain.invoke({"input_documents": documents, "question": query})
answer = response["output_text"]
print("ANSWER :",answer)


ANSWER : The Transformer model generates descriptive nepali captions based on the features extracted from the image


In [39]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [79]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)


vectordb = Chroma.from_documents(
  documents,
  embedding=embeddings,
  persist_directory='./data'
)
vectordb.persist()

In [94]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True
)

result = qa_chain.invoke({'query': 'what does the CNN do?'})
print(result['result'])

Image Processing


In [42]:
from langchain.chains import ConversationalRetrievalChain


qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 6}),
    return_source_documents=True
)

In [95]:
import sys

chat_history = []
user_info = {
    'data': []
}


In [98]:
def update_user_info(name, phone_number, email):
    user_info['data'].append({
        'name': name,
        'phone_number': phone_number,
        'email': email
    })

name = input("Your Name: ")
phone_number = input("Your Phone Number: ")
email = input("Your Email: ")

# Update user info
update_user_info(name, phone_number, email)

print("Now you can ask queries about your documents and type 'q' for exiting. Thank you for your cooperation ")

while True:
    query = input('Prompt: ')

    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()

    # Process the call request with the user information
    if "call" in query.lower():
        print(f"Calling {user_info['data']['name']} at {user_info['data']['phone_number']}...")
        chat_history['Calling'] = f"{user_info['data']['name']}, {user_info['data']['phone_number']}, {user_info['data']['email']}"
    else:
        result = qa_chain({'query': query, 'chat_history': chat_history})
        print('Answer: ' + result['result'])
        chat_history.append((query, result['result']))


Your Name: Sakshyam
Your Phone Number: 9841966438
Your Email: sakain2@gmail.com
Now you can ask queries about your documents and type 'q' for exiting. Thank you for your cooperation 
Prompt: What does tts do?


Answer: This TTS model converts the textual input into synthesized speech
Prompt: Why is django used?


Answer: Web Interface
Prompt: q
Exiting


SystemExit: 

In [99]:
    user_info

{'data': [{'name': 'Samikshya',
   'phone_number': '9841267496',
   'email': 'chapagainsamikshya3@gmail.com'},
  {'name': 'Sakshyam',
   'phone_number': '9841966438',
   'email': 'sakain2@gmail.com'}]}